In [4]:
import pandas as pd
import PySimpleGUI as sg
import datetime

sg.theme('LightBlue5')
layout = [[sg.Text("Đăng ký thông tin thẻ thư viện TPHCM", background_color = "pink", text_color = "green")],
          [sg.Text("Họ và tên", size = 20), sg.Input(key="ten")],
          [sg.Text("Giởi tính", size = 20), sg.Combo(["Nam","Nữ"], key = "gioitinh")],
          [sg.Text("CCCD/CMND", size = 20), sg.Input(key= "CCCD")],
          [sg.Text("Số điện thoại", size = 20), sg.Input(key= "SDT")],
          [sg.Text("ngày đăng ký", size = 20), 
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(31,0,-1)],key ="ngay",size = 5),
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(12,0,-1)],key = "thang",size = 5), 
          sg.Combo([str(i) for i in range(2030,2000,-1)] ,key = "nam",size = 5)], 
          [sg.Text("địa chỉ", size = 20), sg.Multiline(key = "diachi", size = (45,5))],
          [sg.Button("Lưu thông tin", button_color="Violet"),sg.Button("Kiểm tra", button_color="SpringGreen"),
          sg.Button("Thoát", button_color="black"),sg.Button("xóa", button_color="DeepPink")]]

window = sg.Window('Mẫu đăng ký', layout)
dt = pd.read_excel("data.xlsx",dtype=str)

def delete_input():
    for key in values:
        if key != "ngay_dang_ky" and key !=  "gia_tri_the":
            window[key]("")
    return None

def check_input():
    for key in values: 
        if key != "ngay_dang_ky" and key !=  "gia_tri_the" :
            if values[key] == "":
                sg.popup("Bạn Vui lòng nhập đủ thông tin")
                return False
    return True

def check_input_ngay():
    try: datetime.datetime.strptime(values["ngay_dang_ky"], "%d/%m/%Y")

    except:
        delete_input()
        return False
    else:
        return True
    
def check_input_CCCD():
    if len(values["CCCD"]) != 9 and len(values["CCCD"]) != 12:
        sg.popup("Sai thông tin CCCD/CMND")
        delete_input()
        return False
    return True
def check_SDT():
    if len(values["SDT"]) != 10:
        sg.popup("Số điện thoại không đúng")
        delete_input()
        return False
    return True
    

from datetime import date
today = date.today()
    
def check_CCCD():
    if values["CCCD"] in dt["CCCD"].to_list():
        return True

def check_gia_tr_the():
    if values["CCCD"] in dt["CCCD"].to_list():        
        in_dex1 = dt.loc[dt["CCCD"] == values["CCCD"]].index.to_list()[0]
        dt_key1 = dt.iloc[in_dex1]
        dic1 = dt_key1.to_dict()
        if datetime.datetime.strptime(dic1["gia_tri_the"], "%Y-%m-%d %H:%M:%S") > datetime.datetime.strptime(str(today),"%Y-%m-%d"):
            return True
        return False

def update():                 
    in_dex2 = dt.loc[dt["CCCD"] == values["CCCD"]].index.to_list()[0]
    key_list = list(dt.columns.values)
    for key in key_list:
        dt.loc[in_dex2,key] = values[key]
    return values
                        
while True:    
    event, values = window.read() 
    if event == "Thoát" or event == sg.WIN_CLOSED:
        window.close()
        break
        
    if event == "Lưu thông tin":
        if check_input() == True:
            values["ngay_dang_ky"] = values["ngay"]+"/"+values["thang"]+"/"+values["nam"]
            if check_input_ngay() == True and check_input_CCCD() == True and check_SDT() == True:   
                values["gia_tri_the"] = datetime.datetime.strptime(values["ngay_dang_ky"], "%d/%m/%Y") + datetime.timedelta(days=365)
                if check_CCCD() == True and check_gia_tr_the() == True:
                    sg.popup("Thẻ của bạn vẫn còn giá trị")
                    delete_input()
                elif check_CCCD() == True and check_gia_tr_the() == False:
                    update()
                    sg.popup("Thông tin đã được cập nhật thành công") 
                    delete_input()
                else: 
                    dt = dt.append(values, ignore_index=True)
                    dt.to_excel("data.xlsx",index = False)
                    sg.popup("Thông Tin Của Bạn Đã Được Lưu Thành Công")
                    delete_input()
            else: sg.popup("Thông tin ngày không hợp lệ")
    if event == "Kiểm tra":
        layout = [[sg.Text("Vui lòng nhập CMND/CCCD")],
                    [sg.Text("CMND/CCCD"), sg.InputText(key = "CMND_check")],
                    [sg.Submit(), sg.Cancel()]]
        window_check = sg.Window("Kiểm tra thông tin",layout)
        event1, values1 = window_check.read() 
        tham_so = values1["CMND_check"]
        in_dex_to_dt = dt.loc[dt["CCCD"] == tham_so].index.to_list()
        if in_dex_to_dt == []:
            sg.popup("bạn chưa đăng ký thông tin")
            window_check.close()
        else:
            in_dex = in_dex_to_dt[0]
            dt_key = dt.iloc[in_dex]
            dic = dt_key.to_dict()
            print(dic)
            #ngay = dic['ngay_dang_ky']
            #ngay_het_han = datetime.datetime.strptime(ngay, "%d/%m/%Y") + datetime.timedelta(days=365)
            window_check.close()
            for key, value in dic.items():
                if key != "ngay_dang_ky" and key !=  "gia_tri_the":
                    window[key].update(value)
                if key == 'gia_tri_the':
                    ngay_het_han = dic[key]
            sg.popup("ngày hết hạn của thẻ là: ",ngay_het_han)
    if event == "xóa":
        delete_input()

In [2]:
import pandas as pd
import PySimpleGUI as sg
import datetime

sg.theme('LightBlue5')
layout = [[sg.Text("Đăng ký thông tin thẻ thư viện TPHCM", background_color = "pink", text_color = "green")],
          [sg.Text("Họ và tên", size = 20), sg.Input(key="ten")],
          [sg.Text("Giởi tính", size = 20), sg.Combo(["Nam","Nữ"], key = "gioitinh")],
          [sg.Text("CCCD/CMND", size = 20), sg.Input(key= "CCCD")],
          [sg.Text("Số điện thoại", size = 20), sg.Input(key= "SDT")],
          [sg.Text("ngày đăng ký", size = 20), 
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(31,0,-1)],key ="ngay",size = 5),
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(12,0,-1)],key = "thang",size = 5), 
          sg.Combo([str(i) for i in range(2030,2000,-1)] ,key = "nam",size = 5)], 
          [sg.Text("địa chỉ", size = 20), sg.Multiline(key = "diachi", size = (45,5))],
          [sg.Text("Chọn loại hình dịch vụ", size = 20), sg.Combo(["Thé doanh nhân","Thẻ thông thường"], key = "loai")],
          [sg.Button("Lưu thông tin", button_color="blue"),sg.Button("Kiểm tra", button_color="red"),
          sg.Button("Thoát", button_color="black"),sg.Button("xóa", button_color="orange")]]

window = sg.Window('Mẫu đăng ký', layout)
dt = pd.read_excel("data.xlsx",dtype=str)

def delete_input():
    for key in values:
        if key != "ngay_dang_ky" and key !=  "gia_tri_the":
            window[key]("")
    return None

#def kiem_tra_ngay():

from datetime import date
today = date.today()
print(str(today))
 
event, values = window.read() 

if values["CCCD"] in dt["CCCD"].to_list():
    print(True)
else: print(False)


if values["CCCD"] in dt["CCCD"].to_list():        
    in_dex1 = dt.loc[dt["CCCD"] == values["CCCD"]].index.to_list()[0]
    dt_key1 = dt.iloc[in_dex1]
    dic1 = dt_key1.to_dict()
    if dic1["gia_tri_the"] < today:
            print(True)
else: print(False)
print(dt["CCCD"])
print(values["CCCD"])
values["ngay_dang_ky"] = values["ngay"]+"/"+values["thang"]+"/"+values["nam"]
print(datetime.datetime.strptime(values["ngay_dang_ky"], "%d/%m/%Y"))
if event == "Thoát" or event == sg.WIN_CLOSED:
    window.close()
    

2022-08-17
False
False
0    123456741
1    123456963
2    123456965
3    123456875
4    123456987
5    123456975
6    123456971
7          NaN
Name: CCCD, dtype: object

2022-04-30 00:00:00


In [ ]:
123456951

In [2]:
import pandas as pd
import PySimpleGUI as sg

sg.theme('LightBlue5')
layout = [[sg.Text("Đăng ký thông tin thẻ thư viện TPHCM", background_color = "pink", text_color = "green")],
          [sg.Text("Họ và tên", size = 20), sg.Input(key="ten")],
          [sg.Text("Giởi tính", size = 20), sg.Combo(["Nam","Nữ"], key = "gioitinh")],
          [sg.Text("CCCD/CMND", size = 20), sg.Input(key= "CCCD")],
          [sg.Text("Số điện thoại", size = 20), sg.Input(key= "SDT")],
          [sg.Text("ngày đăng ký", size = 20), 
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(31,0,-1)],key ="ngay",size = 5),
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(12,0,-1)],key = "thang",size = 5), 
          sg.Combo([str(i) for i in range(2030,2000,-1)] ,key = "nam",size = 5)], 
          [sg.Text("địa chỉ", size = 20), sg.Multiline(key = "diachi", size = (45,5))],
          [sg.Text("Chọn loại hình dịch vụ", size = 20), sg.Combo(["Thé doanh nhân","Thẻ thông thường"], key = "loai")],
          [sg.Button("Lưu thông tin", button_color="blue"),sg.Button("Kiểm tra", button_color="red"),sg.Button("Thoát", button_color="black")]]

window = sg.Window('Mẫu đăng ký', layout)
dt = pd.read_excel("data.xlsx",dtype=str)

def delete_input():
    for key in values:
        if key != "ngay_dang_ky":
            window[key]("")
    return None

from datetime import date
today = date.today()

while True:    
    event, values = window.read() 
    values["ngay_dang_ky"] = values["ngay"]+"-"+values["thang"]+"-"+values["nam"]
    if event == "Thoát" or event == sg.WIN_CLOSED:
        window.close()
        break
    if event == "Lưu thông tin":        
        dt = dt.append(values, ignore_index=True)
        dt.to_excel("data.xlsx",index = False)
        sg.popup("Thông Tin Của Bạn Đã Được Lưu Thành Công")
        delete_input()
    #if event == "Kiểm tra"
    

{'ten': '1', 'gioitinh': 'Nữ', 'CCCD': '123', 'SDT': '654', 'ngay': '29', 'thang': '12', 'nam': '2028', 'diachi': '123', 'loai': 'Thẻ thông thường', 'ngay_dang_ky': '29-12-2028'}


In [1]:

import datetime
str1= "21/02/13"
d1 = datetime.datetime.strptime(str1, "%y/%m/%d")
print(d1)
print(d1)
#>> 2021-02-13 00:00:00
print(d1 + datetime.timedelta(days=31))
#>> 2021-03-16 00:00:00
print(d1 + datetime.timedelta(days=-2))
#>> 2021-02-11 00:00:00

2021-02-13 00:00:00
2021-02-13 00:00:00
2021-03-16 00:00:00
2021-02-11 00:00:00


In [ ]:
import pandas as pd
import PySimpleGUI as sg
import datetime

sg.theme('LightBlue5')
layout = [[sg.Text("Đăng ký thông tin thẻ thư viện TPHCM", background_color = "pink", text_color = "green")],
          [sg.Text("Họ và tên", size = 20), sg.Input(key="ten")],
          [sg.Text("Giởi tính", size = 20), sg.Combo(["Nam","Nữ"], key = "gioitinh")],
          [sg.Text("CCCD/CMND", size = 20), sg.Input(key= "CCCD")],
          [sg.Text("Số điện thoại", size = 20), sg.Input(key= "SDT")],
          [sg.Text("ngày đăng ký", size = 20), 
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(31,0,-1)],key ="ngay",size = 5),
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(12,0,-1)],key = "thang",size = 5), 
          sg.Combo([str(i) for i in range(2030,2000,-1)] ,key = "nam",size = 5)], 
          [sg.Text("địa chỉ", size = 20), sg.Multiline(key = "diachi", size = (45,5))],
          [sg.Text("Chọn loại hình dịch vụ", size = 20), sg.Combo(["Thé doanh nhân","Thẻ thông thường"], key = "loai")],
          [sg.Button("Lưu thông tin", button_color="blue"),sg.Button("Kiểm tra", button_color="red"),sg.Button("Thoát", button_color="black")]]

window = sg.Window('Mẫu đăng ký', layout)
dt = pd.read_excel("data.xlsx",dtype=str)

def delete_input():
    for key in values:
        if key != "ngay_dang_ky":
            window[key]("")
    return None

from datetime import date
today = date.today()

while True:    
    event, values = window.read() 
    if event == "Thoát" or event == sg.WIN_CLOSED:
        window.close()
        break
    if event == "Lưu thông tin": 
        values["ngay_dang_ky"] = values["nam"]+"/"+values["thang"]+"/"+values["ngay"]
        dt = dt.append(values, ignore_index=True)
        dt.to_excel("data.xlsx",index = False)
        sg.popup("Thông Tin Của Bạn Đã Được Lưu Thành Công")
        delete_input()
    if event == "Kiểm tra":
        if values["CCCD"] == "":
            layout = [[sg.Text("Vui lòng nhập CMND/CCCD")],
                      [sg.Text("CMND/CCCD"), sg.InputText(key = "CMND_check")],
                      [sg.Submit(), sg.Cancel()]]
            window_check = sg.Window("Kiểm tra thông tin",layout)
            event1, values1 = window_check.read() 
            tham_so = values1["CMND_check"]
            in_dex = dt.loc[dt["CCCD"] == tham_so].index.to_list()[0]
            if in_dex == "":
                print("bạn chưa đăng ký thông tin")
            else:
                dt_key = dt.iloc[in_dex]
                dic = dt_key.to_dict()
                print(dic)
                ngay = dic['ngay_dang_ky']
                print(ngay)
                ngay_het_han = datetime.datetime.strptime(ngay, "%Y/%m/%d") + datetime.timedelta(days=365)
                print(ngay_het_han)

{'ten': 'Nguyễn Thị An', 'gioitinh': 'Nữ', 'CCCD': '123456951', 'SDT': '0123456789', 'ngay': '30', 'thang': '10', 'nam': '2021', 'diachi': '20 Nam Kỳ Khởi Nghĩa', 'loai': 'Thé doanh nhân', 'ngay_dang_ky': '2021/10/30'}
2021/10/30
2022-10-30 00:00:00


In [2]:
import pandas as pd
import PySimpleGUI as sg
import datetime
sg.theme('LightBlue5')
layout = [[sg.Text("Đăng ký thông tin thẻ thư viện TPHCM", background_color = "pink", text_color = "green")],
          [sg.Text("Họ và tên", size = 20), sg.Input(key="ten")],
          [sg.Text("Giởi tính", size = 20), sg.Combo(["Nam","Nữ"], key = "gioitinh")],
          [sg.Text("CCCD/CMND", size = 20), sg.Input(key= "CCCD")],
          [sg.Text("Số điện thoại", size = 20), sg.Input(key= "SDT")],
          [sg.Text("ngày đăng ký", size = 20), 
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(31,0,-1)],key ="ngay",size = 5),
          sg.Combo([("0" + str(i) if len(str(i))<2 else str(i)) for i in range(12,0,-1)],key = "thang",size = 5), 
          sg.Combo([str(i) for i in range(2030,2000,-1)] ,key = "nam",size = 5)], 
          [sg.Text("địa chỉ", size = 20), sg.Multiline(key = "diachi", size = (45,5))],
          [sg.Text("Chọn loại hình dịch vụ", size = 20), sg.Combo(["Thé doanh nhân","Thẻ thông thường"], key = "loai")],
          [sg.Button("Lưu thông tin", button_color="blue"),sg.Button("Kiểm tra", button_color="red"),
          sg.Button("Thoát", button_color="black"),sg.Button("xóa", button_color="orange")]]

window = sg.Window('Mẫu đăng ký', layout)
dt = pd.read_excel("data.xlsx",dtype=str)

def check_CCCD():
    if values["CCCD"] in dt["CCCD"]:
        return True
window.read() 


NameError: name 'values' is not defined